# 03 - Model Deployment

The purpose of this notebook is to execute a CI/CD routine to test and deploy the trained model to AI Platform Prediction, using Cloud Build.
The CI/CD rountine is defined in the [model-deployment.yaml](model-deployment.yaml) file, and consists of the following steps:
1. Load and test the the trained model interface.
2. Create and endpoint in AI Platform Prediction if it doesn't exists.
3. Deploy the model to the endpoint.
4. Test the endpoint.


## Setup

In [13]:
import os
import logging

logging.getLogger().setLevel(logging.INFO)

In [14]:
PROJECT = 'ksalama-cloudml'
REGION = 'us-central1'
MODEL_DISPLAY_NAME = 'chicago_taxi_tips_classifier_v1'
ENDPOINT_DISPLAY_NAME = 'chicago_taxi_tips_classification'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

REPO_URL = "https://github.com/ksalama/ucaip-labs.git"
BRANCH = "mb-sdk"

## Build CI/CD  Container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying the model will be executed.

In [ ]:
!echo $CICD_IMAGE_URI

In [ ]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

## 1. Run CI/CD Steps locally

In [15]:
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['MODEL_DISPLAY_NAME'] = MODEL_DISPLAY_NAME
os.environ['ENDPOINT_DISPLAY_NAME'] = ENDPOINT_DISPLAY_NAME

### Run the model artifact testing

In [34]:
!py.test src/tests/model_deployment_tests.py::test_model_artifact -s

============================= test session starts ==============================
platform linux -- Python 3.7.10, pytest-6.2.4, py-1.10.0, pluggy-0.13.1
rootdir: /home/jupyter/ucaip-labs
collecting ... 2021-05-28 15:13:25.269865: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
collected 1 item                                                               

src/tests/model_deployment_tests.py 2021-05-28 15:13:27.901408: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-05-28 15:13:27.901686: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-05-28 15:13:27.901708: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] f

### Run create endpoint

In [38]:
!python build/utils.py \
    --mode=create-endpoint\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}

INFO:google.cloud.bigquery.opentelemetry_tracing:This service is instrumented using OpenTelemetry.OpenTelemetry could not be imported; pleaseadd opentelemetry-api and opentelemetry-instrumentationpackages in order to get BigQuery Tracing data.
2021-05-28 15:19:18.459550: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:Creating endpoint chicago_taxi_tips_classification
INFO:root:Uniquness of objects in project:ksalama-cloudml region:us-central1 validated.
INFO:root:Vertex AI client initialized.
INFO:root:Endpoint is ready.
INFO:root:name: "projects/900786220115/locations/us-central1/endpoints/1622474542318354432"
display_name: "chicago_taxi_tips_classification"
etag: "AMEw9yNpXCUfkco-lGfHfP-Yr-jyexhY9K68k6-G_DFb2SpNqffbwRuqC0KJZWim5Ypt"
create_time {
  seconds: 1622215109
  nanos: 940519000
}
update_time {
  seconds: 1622215110
  nanos: 934797000
}

INFO:root:<google.cloud.aiplatform.models.Endpoint object a

### Run deploy model

In [45]:
!python build/utils.py \
    --mode=deploy-model\
    --project={PROJECT}\
    --region={REGION}\
    --endpoint-display-name={ENDPOINT_DISPLAY_NAME}\
    --model-display-name={MODEL_DISPLAY_NAME}

INFO:google.cloud.bigquery.opentelemetry_tracing:This service is instrumented using OpenTelemetry.OpenTelemetry could not be imported; pleaseadd opentelemetry-api and opentelemetry-instrumentationpackages in order to get BigQuery Tracing data.
2021-05-28 15:36:40.278441: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
INFO:root:serving resources: {'traffic_percentage': 100, 'machine_type': 'n1-standard-2', 'min_replica_count': 1, 'max_replica_count': 1, 'accelerator_type': None, 'accelerator_count': None}
INFO:root:Deploying model chicago_taxi_tips_classifier_v1 to endpoint chicago_taxi_tips_classification
INFO:root:Uniquness of objects in project:ksalama-cloudml region:us-central1 validated.
INFO:root:Vertex AI client initialized.
INFO:google.cloud.aiplatform.models:Deploying Model projects/900786220115/locations/us-central1/models/1544435605025325056 to Endpoint : projects/900786220115/locations/us-central1/endpoin

### Run model endpoint testing

In [ ]:
!py.test src/tests/model_deployment_tests.py::test_model_endpoint

## 2. Execute the Model Deployment CI/CD rountine in Cloud Build

In [ ]:
SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_ENDPOINT_DISPLAY_NAME={ENDPOINT_DISPLAY_NAME},\
"""

!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit --no-source --config build/model-deployment.yaml --substitutions {SUBSTITUTIONS} --timeout=30m